Smart Sports Analyzer (Google Colab Version)
⚽ An intelligent sports match analyzer that uses YOLOv8 for real-time player and ball detection on football (soccer) match videos.
🧠 Built for sports analytics, event tracking, and visual insights — right inside your browser using Google Colab.

🔍 What This Project Does
Detects players (class: person) in every frame of the match video

Annotates the video with bounding boxes and confidence scores

Outputs an analyzed video ready for download

Optionally extendable with DeepSORT for player tracking, pose detection, heatmaps, and ball possession

🛠️ Tools & Tech
Technology	Purpose
YOLOv8 (Ultralytics)	Object detection (players/ball)
OpenCV	Video frame handling & annotations
Google Colab	Cloud execution & easy sharing

-> Google gemini will give real time output

In [4]:
# 📘 Smart Sports Analyzer - Google Colab
# Real-time football match commentary using YOLOv8 + Gemini LLM

# 🧰 Step 1: Install Required Packages
!pip install ultralytics opencv-python google-generativeai --quiet

In [5]:
# 🎥 Step 2: Download a Football Match from YouTube

# 👉 Paste your video URL below
video_url = "https://www.youtube.com/watch?v=VIDEO_ID"  # ← Replace with your link

# Download the video using yt-dlp
!yt-dlp -f 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' "{video_url}" -o "match.mp4"

video_path = "match.mp4"
print(f"✅ Video downloaded and saved as: {video_path}")


[youtube:truncated_id] Extracting URL: https://www.youtube.com/watch?v=VIDEO_ID
ERROR: [youtube:truncated_id] VIDEO_ID: Incomplete YouTube ID VIDEO_ID. URL https://www.youtube.com/watch?v=VIDEO_ID looks truncated.
✅ Video downloaded and saved as: match.mp4


In [2]:
# 🧠 Step 3: Load YOLOv8 and Detect Players Frame-by-Frame
from ultralytics import YOLO
import cv2

model = YOLO('yolov8n.pt')  # You can change to yolov8s.pt for more accuracy

def extract_context(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    context_list = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 10 != 0:  # Skip frames to reduce load
            continue

        results = model(frame)
        frame_context = []

        for r in results:
            for box in r.boxes:
                cls = int(box.cls[0])
                if cls == 0:  # Only detect persons
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    frame_context.append({"bbox": [x1, y1, x2, y2]})

        context_list.append(frame_context)

    cap.release()
    return context_list

contexts = extract_context(video_path)
print(f"✅ Extracted {len(contexts)} context frames.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.4 MB/s eta 0:00:00
[youtube] Extracting URL: https://www.youtube.com/watch?v=kbrDf3tLrZ8
[youtube] kbrDf3tLrZ8: Downloading webpage
[youtube] kbrDf3tLrZ8: Downloading tv client config
[youtube] kbrDf3tLrZ8: Downloading player fc2a56a5-main
[youtube] kbrDf3tLrZ8: Downloading tv player API JSON
[youtube] kbrDf3tLrZ8: Downloading ios player API JSON
ERROR: [youtube] kbrDf3tLrZ8: Video unavailable


In [ ]:
# 🤖 Step 4: Connect to Gemini for Commentary
import google.generativeai as genai

genai.configure(api_key="YOUR_GEMINI_API_KEY")  # ← Replace this with your key
model = genai.GenerativeModel("gemini-1.5-pro")

def get_commentary(frame_context, last_line=""):
    prompt = f'''
You are a football match commentator.
Based on the following detected player bounding boxes, generate a 1-line live commentary.

Player bounding boxes: {frame_context}
Previous line: "{last_line}"

Example output:
"Player 7 dribbles forward on the left wing."
"Ball is passed near the penalty box."
'''
    try:
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception:
        return "Commentary loading..."


In [ ]:
# 🗣️ Step 5: Run Real-time Commentary
import time

print("🎥 Generating match commentary...\n")
last_line = ""

for i, context in enumerate(contexts[:25]):  # Adjust range for longer matches
    line = get_commentary(context, last_line)
    print(f"📣 {line}")
    last_line = line
    time.sleep(1)  # Simulated real-time
